In [29]:
import json, os, re, requests, sys, time
import pandas as pd
import logging
import logging.handlers

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from datetime import date, datetime
from pykrx import stock
from webob.compat import urlparse
import lxml
URL_NAVER_MARKETINDEX_OIL_QUERY = "https://finance.naver.com/marketindex/oilDetail.naver?marketindexCd=%s&fdtc=2"


In [48]:
url_dict = {'휘발유': 'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=%s',
            '고급휘발유':'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_HGSL&page=%s',
            '경유':'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_LO&page=%s',
            '두바이유':'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_DU&fdtc=2&page=%s',
            '브렌트유':'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_BRT&fdtc=2&page=%s',
            'WTI(서부텍사스유)':'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_CL&fdtc=2&page=%s'
            }

In [49]:
url_dict.keys()

dict_keys(['휘발유', '고급휘발유', '경유', '두바이유', '브렌트유', 'WTI(서부텍사스유)'])

In [50]:
url_dict.values()

dict_values(['https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=%s', 'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_HGSL&page=%s', 'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_LO&page=%s', 'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_DU&fdtc=2&page=%s', 'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_BRT&fdtc=2&page=%s', 'https://finance.naver.com/marketindex/worldDailyQuote.naver?marketindexCd=OIL_CL&fdtc=2&page=%s'])

In [131]:
for key in url_dict.keys():
    
    date = []
    values = []
    
    page = 1
    
    print(key)
    
    while page < 2:
        r_url = url_dict[key] % i
        print('r_url = ',r_url)
        r = requests.get(r_url)
        soup = bs(r.content,'lxml')
        tbody = soup.find('tbody')
        tr = tbody.find_all('tr')
        # print(tbody.text.strip('\t\n\r'))
        # print(len(tbody.text.strip()))
        # print(tr)
        
        for t in tr:
            print(t)
            # temp_date = t.find("td",class_="date").text.replace('.','-').strip()
            temp_values = t.find("td",class_="num").text
            values.append(temp_values)
            # print(values)
            # date.append(temp_date)
            # values.append(temp_values)
            # # print(temp_values)
            

#         else:
#             temp = DataFrame(values, index = date, columns = [key])
                
#             Data = pd.merge(Data,temp, how='outer', left_index=True, right_index=True)        
                
#             print(key + '자료 수집 완료')
#             time.sleep(10)
#             break


            
        page += 1
    
    # print(date)
    print(values)
            
            
            
#             temp = DataFrame(value, index = date, columns = [key])
                
#                 Data = pd.merge(Data,temp, how='outer', left_index=True, right_index=True)        
                
#                 print(key + '자료 수집 완료')
#                 time.sleep(10)
#                 break


                


            

휘발유
r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=1
<tr class="down">
<td class="date">
		
		2022.04.01
		</td>
<td class="num">1,996.48</td>
<td class="num"><img alt="하락" height="6" src="https://ssl.pstatic.net/static/nfinance/ico_down.gif" width="7"/> 1.91</td>
<td class="num">
		-0.10%</td>
</tr>
<tr class="down">
<td class="date">
		
		2022.03.31
		</td>
<td class="num">1,998.22</td>
<td class="num"><img alt="하락" height="6" src="https://ssl.pstatic.net/static/nfinance/ico_down.gif" width="7"/> 1.22</td>
<td class="num">
		-0.06%</td>
</tr>
<tr class="down">
<td class="date">
		
		2022.03.30
		</td>
<td class="num">1,999.34</td>
<td class="num"><img alt="하락" height="6" src="https://ssl.pstatic.net/static/nfinance/ico_down.gif" width="7"/> 1.11</td>
<td class="num">
		-0.06%</td>
</tr>
<tr class="down">
<td class="date">
		
		2022.03.29
		</td>
<td class="num">2,000.48</td>
<td class="num"><img alt="하락" height="6" src="https://ssl.pstat

In [284]:
key = '휘발유'


values=[]
i = 1
r_url = url_dict[key] % i
print('r_url = ',r_url)
r = requests.get(r_url)
soup = bs(r.text,'lxml')
data = soup.find('tbody').find_all('tr')
# print(len(data.text.strip()))
# print(data)

for t in data:
    x = t.text.strip().replace('+','').replace('\t','').split('\n')
    x = ' '.join(x).split()
    values.append(x)
    # print(t[i])
    # temp_date = t.find("td",class_="date").text.replace('.','-').strip()
    # temp_values = r.text.strip()
    # print(temp_values)
    # values.append(temp_values)
    # print(values)
    # date.append(temp_date)
    # values.append(temp_values)
print(values)
pd.DataFrame(values)

r_url =  https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=1
[['2022.04.01', '1,996.48', '1.91', '-0.10%'], ['2022.03.31', '1,998.22', '1.22', '-0.06%'], ['2022.03.30', '1,999.34', '1.11', '-0.06%'], ['2022.03.29', '2,000.48', '0.67', '-0.03%'], ['2022.03.28', '2,001.16', '0.31', '0.02%'], ['2022.03.27', '2,000.87', '0.22', '0.01%'], ['2022.03.26', '2,000.70', '1.22', '-0.06%']]


,0,1,2,3
0,2022.04.01,"1,996.48",1.91,-0.10%
1,2022.03.31,"1,998.22",1.22,-0.06%
2,2022.03.30,"1,999.34",1.11,-0.06%
3,2022.03.29,"2,000.48",0.67,-0.03%
4,2022.03.28,"2,001.16",0.31,0.02%
5,2022.03.27,"2,000.87",0.22,0.01%
6,2022.03.26,"2,000.70",1.22,-0.06%


In [13]:
df['OIL_KR_NAME']

0            휘발유
1          고급휘발유
2             경유
3           두바이유
4           브렌트유
5    WTI(서부텍사스유)
Name: OIL_KR_NAME, dtype: object

In [16]:
page = 1

r_url = df['OIL_URL'][0]

In [17]:
r_url

'https://finance.naver.com/marketindex/oilDailyQuote.naver?marketindexCd=OIL_GSL&page=%s'